# Scratch

Figure out how much the sequence length varies in the data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
test = pd.read_csv('/scratch/gpfs/aa8417/QCB557_project/data/test.csv')
train = pd.read_csv('/scratch/gpfs/aa8417/QCB557_project/data/train.csv')

In [ ]:
test_seqs = test['sequence'].tolist()
train_seqs = train['sequence'].tolist()

In [ ]:
seqs = test_seqs + train_seqs

In [ ]:
def get_seq_lengths(sequences):
    seq_lengths = []
    for sequence in sequences:
        length = len(sequence)
        seq_lengths.append(length)
    return seq_lengths

lengths = get_seq_lengths(seqs)

In [ ]:
plt.hist(lengths, bins=20)
plt.xlabel('length of DNA sequences in train and test set')
plt.ylabel('frequency')
plt.title('histogram of DNA sequence lengths')

plt.show()

In [ ]:
min(lengths)

In [ ]:
max(lengths)

So, sequence lengths range from 290 to 500. So max_length in tokenizer should be 500.

### Code bank

In [ ]:
#DON'T NEED THIS ANYMORE
#change classification head of the model
#reference: https://discuss.huggingface.co/t/how-do-i-change-the-classification-head-of-a-model/4720/3
class BinaryDNABERT2Model(nn.Module):
    def __init__(self):
        super(BinaryDNABERT2Model, self).__init__()

        self.base_model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config).to(device)
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(768, 2) # output features from bert is 768 and 2 is number of labels
        
    def forward(self, input_ids, attn_mask):
        outputs = self.base_model(input_ids, attention_mask=attn_mask)
        # You write you new head here
        outputs = self.dropout(outputs[0])
        outputs = self.linear(outputs)
        
        return outputs

In [ ]:
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.Tensor(labels).long()
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

## Attention weight stuff

In [1]:
import pandas as pd
import torch
from transformers import (
    AutoTokenizer, 
    AutoModel, 
    AutoModelForSequenceClassification, 
    DataCollatorForTokenClassification, 
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer, 
    DataCollatorForLanguageModeling
)
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from transformers.models.bert.configuration_bert import BertConfig 
import numpy as np
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
#from training_args_module import training_args
import transformers
import os

import csv
from Bio import SeqIO


import argparse

/home/aa8417/.conda/envs/dna/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
princeton_id = 'aa8417'
project_dir = f'/scratch/gpfs/{princeton_id}/QCB557_project'

model_name = 'fine_tune_new_v3'
model_out_dir = f'{project_dir}/models/{model_name}'

# use gpu
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [8]:
config = BertConfig.from_pretrained(f'{model_out_dir}/config.json', output_attentions=True)
print(config.num_labels) #2 labels
model = AutoModel.from_pretrained(model_out_dir, trust_remote_code=True, config=config)
model.to(device)
'''
config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M", output_hidden_states=True, output_attentions=True)
print(config.num_labels) #2 labels
model = AutoModelForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config)
model.to(device)
'''
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, padding=True)
tokenizer.pad_token = "X"

2


Some weights of the model checkpoint at /scratch/gpfs/aa8417/QCB557_project/models/fine_tune_new_v3 were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
assert config.output_attentions == True

In [9]:
sequence = 'TCAATCACTCTAAGAAGAAACAATCTAGAAAACTAATTAAATCGAGAAGACAAGATTTCACCACTAATGTAGAAATAAACCCTATTAACAAGAACTTGTATTTCCCACACATTTCTGCTGTTCAAATTTTTGACTTCTATAAAAATGAGCAAGTTAACTATCAGTATTTAACATCAGGTGTCAACAATTCTATGGGTAAAGTTAGATTTGAACTGAATTTACAAGACCCAATAATAACTGATTTGAAGTTCACCAAAGATGGGCAATGGATGATTACATACGAAATTGAGTATCCGCCAAATGACCTCTTATCTTCCAAGGACTTAACTCATATCTTGAAATTTTGGACCAAAAACGATAATGAGACAAATTGGAATTTGAAAACGAAAGTAATAAATCCACACGGGATAAGTGTCCCAATTACCAAGATATTGCCTTCACCAAGATCAGTTAATAATAGTCAAGGCTGTTTAACGGCTGACAACAACGGTGGACTGAAA'

In [12]:
def get_model_output(model, tokenizer, sequence, device):
    inputs = tokenizer(sequence, padding='max_length', max_length=128, return_tensors='pt').to(device)
    input_ids = inputs['input_ids'].to(device)
    print(input_ids)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_attentions=True)
        print(outputs)
        print(outputs.hidden_states.shape)

    predicted_probs = torch.softmax(outputs.logits, dim=-1)
    attention_weights = outputs.attentions
    print(np.array(attention_weights).shape)

    predicted_label = torch.argmax(predicted_probs, dim=-1).item()

    return predicted_label, predicted_probs, attention_weights

predicted_label, predicted_probs, attention_weights = get_model_output(model, tokenizer, sequence, device)
print("Predicted Label:", predicted_label)
print("Predicted Probabilities:", predicted_probs)
print("Attention Weights:", attention_weights)


tensor([[   1,   56, 2366,  451, 1304,   88,   85,   79,  364,   16,  145,  225,
           73,  517,   79,  106, 1903,  338,   84,  466,   29,  160, 1608, 3076,
          146, 2436,  554,   37,  125, 1838,  153, 1180,  160,   20,  197,  103,
           56,  162, 3903, 3614, 3917,  119,   94,   27,  158,   27,  228,  459,
           52,  448,  131,   83,  328,  108,  585,  104, 3403,  282,   26,  463,
          528,  154,   47, 3918,  150,   63,  723, 2415,   33,  718,   93,   20,
          199,  321,   57, 2597,  409,   68,   38,  401,   72,  250, 1094,  581,
           76,  657, 2444,   76,  254,  153,  137,  168, 3658, 2846,   59,  361,
           72,  108,  357,    2,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]], device='cuda:0')
(tensor([[[ 0.1051, -0.0844, -0.0708,  ..., -0.1008,  0.3531, -0.0425],
         [-0.0933,  0.1621, -0.1887,  ...,

AttributeError: 'tuple' object has no attribute 'hidden_states'

In [ ]:
config.output_attentions

In [ ]:
print(model)

In [ ]:
outputs.hidden_state

In [7]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertUnpadAttention(
            (self): BertUnpadSelfAttention(
              (dropout): Dropout(p=0.0, inplace=False)
              (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (mlp): BertGatedLinearUnitMLP(
            (gated_layers): Linear(in_features=768

In [ ]:
idntity = testing_indx_0/1
sequence
input_ids
attention_weights
embedding 